In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import hist, processor
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt

def get_sv_in_jet(jet, sv, R=0.8):

    sv_dr = jet.delta_r(sv.p4)
    sv_in_jet = sv_dr < R

    return sv_in_jet

/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
filename = "/pnfs/psi.ch/cms/trivcat/store/user/mmarcheg/BTVNanoCommissioning/store/user/mmarcheg/PFNano/QCD_Pt-170To300_MuEnrichedPt5_TuneCP5_13TeV-pythia8/RunIISummer20UL16PreVFP-v01-106X_mcRun2_asymptotic_v17-v1/220311_144646/0000/nano_mc2016pre_162.root"
#filename = "/pnfs/psi.ch/cms/trivcat/store/user/mmarcheg/BTVNanoCommissioning/store/user/mmarcheg/PFNano/QCD_Pt-1000_MuEnrichedPt5_TuneCP5_13TeV-pythia8/RunIISummer20UL16PreVFP-106X_mcRun2_asymptotic_v17-v1/211213_173836/0000/nano_mc2016pre_3-7.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=10000).events()

/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJet_btagDDBvLV2 in <TTree 'Events' (1612 branches) at 0x7ff27c4931c0>, taking first instance
  warnings.warn(
/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJet_btagDDCvBV2 in <TTree 'Events' (1612 branches) at 0x7ff27c4931c0>, taking first instance
  warnings.warn(
/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJet_btagDDCvLV2 in <TTree 'Events' (1612 branches) at 0x7ff27c4931c0>, taking first instance
  warnings.warn(
/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJet_nBHadrons in <TTree 'Events' (1612 branches) at 0x7ff27c49

In [3]:
mask = ak.num(events.FatJet) > 0
events = events[mask]

In [4]:
jet1 = events.FatJet[:,0]
jet2 = ak.pad_none(events.FatJet,2)[:,1]
sv   = events.SV
jet2.pt

<Array [183, 185, 188, 217, ... 198, 206, 234] type='9351 * ?float32[parameters=...'>

In [5]:
dr1 = jet1.delta_r(sv.p4)
dr1

<Array [[3.07, 2.93, 3.07, 3.17, ... [3.4]] type='9351 * var * float32'>

In [6]:
dr2 = jet2.delta_r(sv.p4)
dr2

<Array [[0.095, 0.204, 0.114, ... [0.0112]] type='9351 * option[var * float32]'>

In [7]:
dr2

<Array [[0.095, 0.204, 0.114, ... [0.0112]] type='9351 * option[var * float32]'>

In [8]:
dr1[:5].tolist()

[[3.0740432739257812, 2.932978868484497, 3.071471929550171, 3.166590452194214],
 [0.053716354072093964, 0.09597958624362946],
 [],
 [3.659698009490967, 3.6384541988372803],
 []]

In [9]:
dr2[:5].tolist()

[[0.09498176723718643,
  0.2043742835521698,
  0.1139734759926796,
  0.6798751354217529],
 [2.4852826595306396, 2.5709986686706543],
 [],
 [0.04078628495335579, 0.041919734328985214],
 None]

In [64]:
mask1 = get_sv_in_jet(jet1, sv)
mask1[:5].tolist()

[[False, False, False, False], [True, True], [], [False, False], []]

In [66]:
mask2 = get_sv_in_jet(jet2, sv)
empty = ak.Array([ak.count(mask2)*[]])
mask2 = ak.where(ak.is_none(mask2), empty, mask2)
mask2[:5].tolist()

[[True, True, True, True], [False, False], [], [True, True], []]

In [67]:
sv_in_jet1 = events.SV[mask1]
sv_in_jet2 = events.SV[mask2]
sv_in_jet1.pt[:5].tolist()

[[], [76.8125, 33.625], [], [], []]

In [68]:
sv_in_jet1.pt[:5].tolist()

[[], [76.8125, 33.625], [], [], []]

In [69]:
sv_in_jet2.pt[:5].tolist()

[[7.16796875, 6.63671875, 6.8828125, 2.869140625],
 [],
 [],
 [9.7265625, 13.34375],
 []]

In [14]:
ak.concatenate((sv_in_jet1, sv_in_jet2), axis=1).pt[:5].tolist()

[[7.16796875, 6.63671875, 6.8828125, 2.869140625],
 [76.8125, 33.625],
 [],
 [9.7265625, 13.34375],
 []]

In [15]:
ak.unflatten(sv_in_jet1, ak.ones_like(jet1.pt, dtype=int)).pt

<Array [[[]], [[76.8, 33.6], ... [[]], [[]]] type='9351 * var * var * float32[pa...'>

In [16]:
m1 = ak.unflatten(mask1, ak.ones_like(ak.num(mask1)))
m2 = ak.unflatten(mask2, ak.ones_like(ak.num(mask2)))
m2

<Array [[[True, True, True, True, ... [[True]]] type='9351 * var * option[var * ...'>

In [24]:
ak.firsts(sv_in_jet1.pt)
ak.ones_like(jet1.pt, dtype=int)

<Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='9351 * int64[parameters={"...'>

In [30]:
sv_in_jet1_unflatten = ak.unflatten(ak.firsts(sv_in_jet1), ak.ones_like(jet1.pt, dtype=int))
sv_in_jet2_unflatten = ak.unflatten(ak.firsts(sv_in_jet2), ak.ones_like(jet1.pt, dtype=int))

In [34]:
ak.concatenate((sv_in_jet1_unflatten, sv_in_jet2_unflatten), axis=1).pt[:5].tolist()

[[None, 7.16796875],
 [76.8125, None],
 [None, None],
 [None, 9.7265625],
 [None, None]]

In [77]:
sv_in_jet1.p4[:5].tolist()

[[],
 [{'pt': 76.8125,
   'eta': 0.207916259765625,
   'phi': 2.07275390625,
   'mass': 3.33984375},
  {'pt': 33.625,
   'eta': 0.150634765625,
   'phi': 2.19091796875,
   'mass': 3.94140625}],
 [],
 [],
 []]

In [61]:
sv_in_jet1.p4.sum()[:5].tolist()

[{'x': 0.0, 'y': 0.0, 'z': 0.0, 't': 0.0},
 {'x': -56.49839782714844,
  'y': 94.70137786865234,
  'z': 21.17015266418457,
  't': 112.78465270996094},
 {'x': 0.0, 'y': 0.0, 'z': 0.0, 't': 0.0},
 {'x': 0.0, 'y': 0.0, 'z': 0.0, 't': 0.0},
 {'x': 0.0, 'y': 0.0, 'z': 0.0, 't': 0.0}]

In [63]:
np.log(sv_in_jet1.p4.sum().mass)[:5].tolist()

/work/mmarcheg/miniconda3/envs/pocket-coffea/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(


[-inf, 2.358278274536133, -inf, -inf, -inf]

In [74]:
sumsv1mass = sv_in_jet1.p4.sum().mass
sumsv2mass = sv_in_jet2.p4.sum().mass
ak.unflatten(sumsv1mass, ak.ones_like(sumsv1mass, dtype=int))

<Array [[0], [10.6], [0], ... [4.42], [0], [0]] type='9351 * var * float32'>

In [75]:
ak.concatenate((sumsv1mass[:,None], sumsv2mass[:,None]), axis=1)

<Array [[0, 7], [10.6, 0, ... 0.996], [0, 1.4]] type='9351 * var * ?float32'>

In [76]:
events.FatJet.pt

<Array [[209, 183], [189, ... 206], [268, 234]] type='9351 * var * float32[param...'>

In [54]:
np.isnan(np.log(ak.where(sumsv1mass == 0, -2, sumsv1mass)))

<Array [True, False, True, ... True, True] type='9351 * bool'>

In [57]:
ak.where(np.isnan(np.log(ak.where(sumsv1mass == 0, -2, sumsv1mass))), )

(<Array [0, 2, 3, 4, ... 9345, 9346, 9349, 9350] type='5103 * int64'>,)

In [84]:
ak.count(sv_in_jet1.pt, axis=1)

<Array [0, 2, 0, 0, 0, 0, ... 0, 0, 1, 2, 0, 0] type='9351 * int64'>